In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
from pathlib import Path
sys.path.insert(0,"..")
from src.evaluation.evaluate import match,match_ignore_case,match_ignore_case_punc,match_ignore_case_punc_keep_hyphen, evaluate_df
from src import alignment
from src.data.make_dataset import depunc, tokenize_skiphyph, recompose
import numpy as np
import re


In [236]:
def match_invariable_order(gold,prediction):
    ''' only penalizes predictions missing from gold, but does not if prediction has more tokens'''
    s1 =str(gold).lower()
    s2 =str(prediction).lower()
    s1 = depunc(s1)
    s2 = depunc(s2)
    s1 = s1.split()
    s2 = s2.split()
    
    l1 = len(s1)
    l2 = len(s2)
    # if gold is just a punc
    if l1 == 0:
       return {'score':1,'len':0,'found':0}
    
    d1 = {}
    d2 = {}
    for t1 in s1:
        if t1 in d1:
            d1[t1] += 1
        else:
            d1[t1] = 1

    for t2 in s2:
        if t2 in d2:
            d2[t2] -= 1
        else:
            d2[t2] = -1

    for t,c in d2.items():
        if t in d1:
            d1[t] -= 1 
    mismatches = sum([x for x in d1.values() if x>0])
    
    return {'score':(l1-mismatches)/l1,'len':l1,'found':l1-mismatches}
 
    


In [237]:
match_invariable_order(dev[dev['sentID']==1]['rom'].values[0],dev[dev['sentID']==1]['p'].values[0])

{'score': 0.3333333333333333, 'len': 6, 'found': 2}

In [2]:
home = Path(os.getcwd()).parent
dev = pd.read_csv(f'{home}/data/processed/dev.tsv',delimiter='\t')


In [179]:
def evaluate_sentences(golddf,gold_column='rom',source_column='ar',predict_series={},extra_columns=['sentID','comb.tag',],eval_funcs=[match,match_ignore_case,match_ignore_case_punc,match_ignore_case_punc_keep_hyphen,match_invariable_order]):
    evaldf = golddf.copy()
    extra_columns.extend([gold_column,source_column])
    evaldf = evaldf[extra_columns]
    for p in predict_series:
        evaldf[p] = predict_series[p]
        
    
    for func in eval_funcs:
        for p in predict_series:
            evaldf[func.__name__] = evaldf.apply(lambda x: func( str(x[p]), str(x[gold_column]) ) ,axis=1)

    return evaldf

In [180]:
home = Path(os.getcwd()).parent.joinpath('predictions_out')
predictions_list = []
for dirpath, dirnames, filenames in os.walk(home):
    for name in filenames:
        if name.endswith('.out') or name.endswith('.tsv'):
            predictions_list.append(name)
sorted(predictions_list,reverse=True)

['rules-simple.tsv',
 'rules-morph.tsv',
 'mle-translit_simple-size1.0.tsv',
 'mle-translit_simple-size0.5.tsv',
 'mle-translit_simple-size0.25.tsv',
 'mle-translit_simple-size0.125.tsv',
 'mle-translit_simple-size0.0625.tsv',
 'mle-translit_simple-size0.03125.tsv',
 'mle-translit_simple-size0.015625.tsv',
 'mle-translit_morph-size1.0.tsv',
 'mle-translit_morph-size0.5.tsv',
 'mle-translit_morph-size0.25.tsv',
 'mle-translit_morph-size0.125.tsv',
 'mle-translit_morph-size0.0625.tsv',
 'mle-translit_morph-size0.03125.tsv',
 'mle-translit_morph-size0.015625.tsv',
 'mle-OOV-size1.0.tsv',
 'mle-OOV-size0.5.tsv',
 'mle-OOV-size0.25.tsv',
 'mle-OOV-size0.125.tsv',
 'mle-OOV-size0.0625.tsv',
 'mle-OOV-size0.03125.tsv',
 'mle-OOV-size0.015625.tsv',
 'line2line-dev.out']

In [239]:
predictions_list = [
#  'rules-simple.tsv',
#  'rules-morph.tsv',
 
 'line2line-dev.out',

#  'mle-translit_simple-size1.0.tsv',
#  'mle-translit_simple-size0.5.tsv',
#  'mle-translit_simple-size0.25.tsv',
#  'mle-translit_simple-size0.125.tsv',
#  'mle-translit_simple-size0.0625.tsv',
#  'mle-translit_simple-size0.03125.tsv',
#  'mle-translit_simple-size0.015625.tsv',

#  'mle-translit_morph-size1.0.tsv',
#  'mle-translit_morph-size0.5.tsv',
#  'mle-translit_morph-size0.25.tsv',
#  'mle-translit_morph-size0.125.tsv',
#  'mle-translit_morph-size0.0625.tsv',
#  'mle-translit_morph-size0.03125.tsv',
#  'mle-translit_morph-size0.015625.tsv',

#  'mle-OOV-size1.0.tsv',
#  'mle-OOV-size0.5.tsv',
#  'mle-OOV-size0.25.tsv',
#  'mle-OOV-size0.125.tsv',
#  'mle-OOV-size0.0625.tsv',
#  'mle-OOV-size0.03125.tsv',
#  'mle-OOV-size0.015625.tsv'
 ]

In [240]:
predictions = {}

for dirpath, dirnames, filenames in os.walk(home):
    for name in filenames:
        if name in predictions_list:
            with open (f'{dirpath}/{name}','r') as p:
                lines = [x.strip() for x in p.readlines()]
                name = name.replace('.tsv','').replace('line2line-dev.out','seq2seq')
                predictions[name] = pd.Series(lines)


In [244]:
dev[dev['match_invariable_order'].apply(lambda x: x['score']<1)][:10][['rom','p','match_invariable_order']].values

array([['Muqaddimāt thawrat 23 Yūlyū sanat 1952 :',
        'Muqaddimāt thawrat 23 Yūlīyū sanat 1952 :',
        {'score': 0.8333333333333334, 'len': 6, 'found': 5}],
       ['al-kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārāt al-muwaẓaffīn, asbāb al-thawrah, Fārūq yumahhid lil-thawrah /',
        'al-Kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārāt al-muwaẓẓafīn, asbāb al-thawrah, fārūq yamhud lil-thawrah /',
        {'score': 0.8333333333333334, 'len': 12, 'found': 10}],
       ['Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlyū sanat 1952',
        'Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlīya sanat 1952',
        {'score': 0.8571428571428571, 'len': 7, 'found': 6}],
       ['Maktabat al-Nahḍah al-Misrīyah,',
        'Maktabat al-Nahḍah al-Miṣrīyah,',
        {'score': 0.6666666666666666, 'len': 3, 'found': 2}],
       ['Manshūrāt al-Īsīskū,', 'Mansūrāt al-Īsīskū,',
        {'score': 0.5, 'len': 2, 'found': 1}],
       ['Bin Muḥammad, ʻAlī.', 'ʻAlī ibn Muḥammad,',
        {'score': 0.6666666

In [242]:
for p in predictions_list:
    p = p.replace('.tsv','').replace('line2line-dev.out','seq2seq')
    dev['p'] = predictions[p]

    dev['match_invariable_order'] = dev.apply(lambda x: match_invariable_order(x['rom'],x['p']),axis=1)
    print(p,end='\t')
    score = dev['match_invariable_order'].apply(lambda x: x['found']).sum()/senteval['match_invariable_order'].apply(lambda x: x['len']).sum()
    print(round(score*100,2))

seq2seq	90.02


In [245]:
dev[['rom','p','match_invariable_order']][:10].values

array([['Rāfiʻī, ʻAbd al-Raḥmān.', 'Rāfiʻī, ʻAbd al-Raḥmān.',
        {'score': 1.0, 'len': 3, 'found': 3}],
       ['Muqaddimāt thawrat 23 Yūlyū sanat 1952 :',
        'Muqaddimāt thawrat 23 Yūlīyū sanat 1952 :',
        {'score': 0.8333333333333334, 'len': 6, 'found': 5}],
       ['al-kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārāt al-muwaẓaffīn, asbāb al-thawrah, Fārūq yumahhid lil-thawrah /',
        'al-Kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārāt al-muwaẓẓafīn, asbāb al-thawrah, fārūq yamhud lil-thawrah /',
        {'score': 0.8333333333333334, 'len': 12, 'found': 10}],
       ['bi-qalam ʻAbd al-Raḥmān al-Rifāʻī.',
        'bi-qalam ʻAbd al-Raḥmān al-Rifāʻī.',
        {'score': 1.0, 'len': 4, 'found': 4}],
       ['Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlyū sanat 1952',
        'Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlīya sanat 1952',
        {'score': 0.8571428571428571, 'len': 7, 'found': 6}],
       ['al-Ṭabʻah 2.', 'al-Ṭabʻah 2.',
        {'score': 1.0, 'len': 2, 'found': 

In [246]:
home = Path(os.getcwd()).parent.joinpath('evaluation/dev')
home
evals = [str(home)+'/'+path for path in os.listdir(home)]
evals = sorted(evals,reverse=True)
evals

['/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/rules-simple.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/rules-morph.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size1.0.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.5.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.25.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.125.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.0625.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.03125.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.015625.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_morph-size1.0.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_morph-size0.5.tsv',
 '/Users/fae211/ba3sasah/LOC_transcribe

In [247]:
# select order
evals = [
    '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/line2line-dev.out.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/rules-simple.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/rules-morph.tsv',

#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-OOV-size1.0.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-OOV-size0.5.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-OOV-size0.25.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-OOV-size0.125.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-OOV-size0.0625.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-OOV-size0.03125.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-OOV-size0.015625.tsv',

#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size1.0.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.5.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.25.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.125.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.0625.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.03125.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_simple-size0.015625.tsv',

#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_morph-size1.0.tsv',
# #  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_morph-size0.5.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_morph-size0.25.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_morph-size0.125.tsv',
#  '/Users/fae211/ba3sasah/LOC_transcribe/evaluation/dev/mle-translit_morph-size0.0625.tsv',
 ]

In [253]:
def score_match(matchcolumn,to_string=True):
    if to_string:
        score = matchcolumn.value_counts(normalize=True).to_string()
        print(matchcolumn.name)
        print(score)
        # logging.info(f'score for: {matchcolumn.name}\n{score}')
    else:
        return matchcolumn.value_counts(normalize=True)

In [254]:
score_match(teste['match'])

match
True     0.869846
False    0.130154


In [ ]:
def evaluate_df(df,predict_columns='all',extra_info_columns='all',eval_funcs='all', allow_misaligned=True):
    ''' unless default all is passed, must pass a list of column names and functions'''
    
    if predict_columns == 'all':
        predict_columns = [column for column in df.columns if column.startswith('predict_')]
        

    if extra_info_columns == 'all':
        extra_info_columns = ['sentID','recID','comb.tag']
    
    if eval_funcs == 'all':
        eval_funcs=[match,match_ignore_case,match_ignore_case_punc,match_ignore_case_punc_keep_hyphen]

    
    evaluation = df.apply(lambda row: evaluate_tokens(gold=row['rom'],source=row['ar'], predictions=dict(map(lambda x: (x,row[x]),predict_columns)), extra_info=dict(map(lambda x: (x,row[x]),extra_info_columns)), allow_misaligned=allow_misaligned)  ,axis=1)

    return pd.concat(list(evaluation))


In [250]:
teste = pd.read_csv(evals[0],delimiter='\t')
# teste[teste['aligned_source/prediction']==False]
teste['match_ignore_case_punc'].value_counts(normalize=True)

True     0.908154
False    0.091846
Name: match_ignore_case_punc, dtype: float64

In [148]:
def print_scores(evals,matches):
    all = {}
    for e in evals:
        scores = {}
        df = pd.read_csv(e,delimiter='\t')
        predict_column = [c for c in df.columns if c.startswith('predict')][0]
        for m in matches:
            score = evaluate.score_match(df[m],to_string=False)[True]
            score = round(score*100,1)
            scores[m] = score
        all[predict_column] = scores
    return pd.DataFrame(all).T

In [152]:
scores = print_scores(evals=evals,matches=['match','match_ignore_case','match_ignore_case_punc'])

KeyError: 'match_ignore_case_punc'

In [150]:
scores

,match,match_ignore_case
predict_rules-simple_tok,16.2,17.3
predict_rules-morph_tok,67.9,84.3
predict_mle-translit_simple-size1.0_tok,84.5,91.5
predict_mle-translit_simple-size0.5_tok,83.2,90.3
predict_mle-translit_simple-size0.25_tok,80.8,88.1
predict_mle-translit_simple-size0.125_tok,76.0,83.6
predict_mle-translit_simple-size0.0625_tok,73.4,80.9
predict_mle-translit_simple-size0.03125_tok,69.2,76.2
predict_mle-translit_simple-size0.015625_tok,64.5,71.3
predict_mle-translit_morph-size1.0_tok,85.3,92.4


In [151]:
scores.to_clipboard()

PosixPath('/Users/fae211/ba3sasah/LOC_transcribe')

In [38]:
seq = pd.read_csv(home.joinpath('evaluation/dev/eval-line2line-dev.out'),delimiter='\t')
mlemorph = pd.read_csv(home.joinpath('evaluation/dev/eval-mle-translit_morph-size1.0.tsv'),delimiter='\t')
rulesmorph = pd.read_csv(home.joinpath('evaluation/dev/eval-rules-morph.tsv'),delimiter='\t')

In [5]:
mlemorph.columns

Index(['sentID', 'comb.tag', 'gold_tok', 'source_tok', 'prediction_tok',
       'aligned_source/gold', 'aligned_source/prediction',
       'aligned_gold/prediction', 'match', 'match_ignore_case',
       'match_ignore_case_punc', 'match_ignore_case_punc_keep_hyphen'],
      dtype='object')

In [29]:
error_analysis = rulesmorph[rulesmorph['match_ignore_case_punc_keep_hyphen']==False]

In [6]:
error_analysis_mle = mlemorph[mlemorph['match_ignore_case_punc_keep_hyphen']==False]

In [30]:
error_analysis[error_analysis['aligned_source/gold']==True]['sentID'].drop_duplicates()

3              1
15             2
27             4
38             6
39            48
           ...  
249777    599415
249804    599456
249815    599457
249819    599508
249830    599556
Name: sentID, Length: 25419, dtype: int64

In [ ]:
rulesmorph[rulesmorph['']]

In [31]:
dev = pd.read_csv(home.joinpath('data/processed/dev.tsv'),delimiter='\t')#.set_index('sentID')
dev = dev.set_index('sentID')

In [32]:
error_analysis['source_sent'] = error_analysis['sentID'].apply(lambda x: dev.loc[x]['ar'])
error_analysis['gold_sent'] = error_analysis['sentID'].apply(lambda x: dev.loc[x]['rom'])

In [25]:
random_sample = error_analysis.sample(100)

In [26]:
random_sample.to_clipboard()

In [33]:
error_analysis[:100].to_clipboard()

In [34]:
dev.loc[227]

rom         ʻAlī al-ʻUraybī.
ar              علي العريبي.
rom_raw     ʻAlī al-ʻUraybī.
ar_raw          علي العريبي.
comb.tag                245c
recID               umich-41
subtag                     c
tag                      245
link                       4
source                 umich
splits                   dev
Name: 227, dtype: object

In [65]:
extracted_lines = pd.read_csv(home.joinpath('data/extracted_lines/extracted_lines.tsv'),delimiter='\t')

In [74]:
extracted_lines[extracted_lines['recID']=='aco-1163'].to_clipboard()

In [75]:
dev[dev['recID']=='aco-1163']#.index

,rom,ar,rom_raw,ar_raw,comb.tag,recID,subtag,tag,link,source,splits
sentID,,,,,,,,,,,


In [53]:
dev[dev['ar'].apply(lambda x: 'كوع' in str(x))]

,rom,ar,rom_raw,ar_raw,comb.tag,recID,subtag,tag,link,source,splits
sentID,,,,,,,,,,,
191649,"Akwaʻ, Ismāʻīl ibn ʻAlī.",أكوع، إسماعيل بن علي.,"Akwaʻ, Ismāʻīl ibn ʻAlī.",‏أكوع، إسماعيل بن علي.,100a,loc-16930,a,100,0,loc,dev
191652,taʼlīf Ismāʻīl ibn ʻAlī al-Akwaʻ.,تأليف إسماعيل بن علي الأكوع.,taʼlīf Ismāʻīl ibn ʻAlī al-Akwaʻ.,‏تأليف إسماعيل بن علي الأكوع.,245c,loc-16930,c,245,4,loc,dev
349612,li-Abī Nuʻaym al-Aṣbahānī ; taḥqīq wa-atakhrīj...,لأبي نعيم الأصبهاني ؛ تحقيق وتخريج طارق محمد س...,li-Abī Nuʻaym al-Aṣbahānī ; taḥqīq wa-at...,لأبي نعيم الأصبهاني ؛ تحقيق وتخريج طارق محمد س...,245c,loc-45036,c,245,4,loc,dev
399097,"Akwaʻ, Muḥammad ibn ʻAlī.",أكوع، محمد بن علي.,"Akwaʻ, Muḥammad ibn ʻAlī.",‏أكوع، محمد بن علي.,100a,loc-54336,a,100,0,loc,dev
399099,Muḥammad ibn ʻAlī al-Akwaʻ.,محمد بن علي الأكوع.,Muḥammad ibn ʻAlī al-Akwaʻ.,‏محمد بن علي الأكوع.,245c,loc-54336,c,245,4,loc,dev
399100,ʻAmīd Muḥammad ibn ʻAlī al-Akwaʻ -- yatadhakka...,عميد محمد بن علي الأكوع ... يتذكر أحداث ثورة ١...,ʻAmīd Muḥammad ibn ʻAlī al-Akwaʻ-- yatadhak...,‏عميد محمد بن علي الأكوع ... يتذكر أحداث ثورة ...,246a,loc-54336,a,246,4,loc,dev
440489,[jamʻ wa-iʻdād] Maḥmūd Muḥammad al-Nākūʻ.,[جمع وإعداد] محمود محمد الناكوع.,[jamʻ wa-iʻdād] Maḥmūd Muḥammad al-Nākūʻ.,[جمع وإعداد] محمود محمد الناكوع.,245c,loc-61607,c,245,4,loc,dev


In [39]:
seq[seq['sentID']==224]

,sentID,comb.tag,gold_tok,source_tok,prediction_tok,aligned_source/gold,aligned_source/prediction,aligned_gold/prediction,match,match_ignore_case,match_ignore_case_punc,match_ignore_case_punc_keep_hyphen
98,224,100a,"ʻUraybī,",علي,ʻAlī,True,True,True,False,False,False,False
99,224,100a,ʻAlī.,العريبي.,al-ʻUraybī.,True,True,True,False,False,False,False


In [178]:
predictions = pd.read_table(home.parents[1].joinpath('predictions_out/mle-backoff_translit_morph/dev/mle-translit_morph-size1.0.tsv'),header=None)

In [193]:
testpredict = predictions.loc[41][0]

'taʼlīf Abī Jaʻfar ibn Muḥammad ibn Ismāʻīl al-Naḥḥās ; dirāsah wa-taḥqīq Sulaymān ibn Ibrāhīm ibn ʻAbd Allāh al-Lāḥim.'

In [197]:
alignment.align_wordsDF(predictions.loc[41][0],dev.loc[41]['rom'],bckf=False)

,0,1,2,3
0,taʼlīf,taʼlīf,n,0
1,Abī,Abī,n,0
2,Jaʻfar,Jaʻfar,n,0
3,_,Aḥmad,i,1
4,ibn,ibn,n,1
5,Muḥammad,Muḥammad,n,1
6,ibn,ibn,n,1
7,Ismāʻīl,Ismāʻīl,n,1
8,al-Naḥḥās,al-Naḥḥās,n,1
9,;,;,n,1


In [194]:
error_analysis[error_analysis['sentID']==572]['source_sent'].drop_duplicates()

163    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
164    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
165    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
166    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
167    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
168    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
169    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
170    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
171    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
172    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
173    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
174    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
175    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
176    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
177    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
178    تأليف أبي جعفر بن محمد بن إسماعيل النحاس ؛ درا...
Name: source_sent, dtype: object

In [165]:
testeval = mlemorph[mlemorph['sentID']==572]
testeval

,gold_tok,source_tok,aligned_source/gold,predict_mle-translit_morph-size1.0_tok,sentID,recID,comb.tag,aligned_source/prediction,match,match_ignore_case
160,taʼlīf,تأليف,False,taʼlīf,572,umich-71,245c,True,True,True
161,Abī,أبي,False,Abī,572,umich-71,245c,True,True,True
162,Jaʻfar,جعفر,False,Jaʻfar,572,umich-71,245c,True,True,True
163,Aḥmad,بن,False,ibn,572,umich-71,245c,True,False,False
164,ibn,محمد,False,Muḥammad,572,umich-71,245c,True,False,False
165,Muḥammad,بن,False,ibn,572,umich-71,245c,True,False,False
166,ibn,إسماعيل,False,Ismāʻīl,572,umich-71,245c,True,False,False
167,Ismāʻīl,النحاس,False,al-Naḥḥās,572,umich-71,245c,True,False,False
168,al-Naḥḥās,؛,False,;,572,umich-71,245c,True,False,False
169,;,دراسة,False,dirāsah,572,umich-71,245c,True,False,False


In [198]:
error_analysis

,gold_tok,source_tok,aligned_source/gold,predict_mle-translit_morph-size1.0_tok,sentID,recID,comb.tag,aligned_source/prediction,match,match_ignore_case,source_sent,gold_sent
6,Yūlyū,يوليوا,True,Ywlywā,1,umich-1,245a,True,False,False,مقدمات ثورة 23 يوليوا سنة 1952 :,Muqaddimāt thawrat 23 Yūlyū sanat 1952 :
15,wazārāt,وزارات,True,wizārāt,2,umich-1,245b,True,False,False,الكفاح في القتال، حريق القاهرة، وزارات الموظفي...,"al-kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārā..."
16,"al-muwaẓaffīn,",الموظفين،,True,"al-muwaẓẓafīn,",2,umich-1,245b,True,False,False,الكفاح في القتال، حريق القاهرة، وزارات الموظفي...,"al-kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārā..."
29,thalāthah,ثلاثة,True,thalāthat,4,umich-1,246a,True,False,False,مقدمات ثورة ثلاثة وعشرين يوليوا سنة 1952,Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlyū ...
31,Yūlyū,يوليوا,True,Ywlywā,4,umich-1,246a,True,False,False,مقدمات ثورة ثلاثة وعشرين يوليوا سنة 1952,Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlyū ...
...,...,...,...,...,...,...,...,...,...,...,...,...
454,Ibrāhīm,محمد,True,Muḥammad,1690,umich-201,100a,True,False,False,إبراهيم محمد سلقيني.,"Salqīnī, Ibrāhīm Muḥammad."
455,Muḥammad.,سلقيني.,True,Salqīnī.,1690,umich-201,100a,True,False,False,إبراهيم محمد سلقيني.,"Salqīnī, Ibrāhīm Muḥammad."
472,"Jabbūr,",جبرايل,True,Jbrāyl,1770,umich-211,100a,True,False,False,جبرايل سليمان جبور.,"Jabbūr, Jibrāʼīl Sulaymān."
473,Jibrāʼīl,سليمان,True,Sulaymān,1770,umich-211,100a,True,False,False,جبرايل سليمان جبور.,"Jabbūr, Jibrāʼīl Sulaymān."


In [305]:
score_invariable_order('ya mama guli lisalaam','ya baba latguuls lisalaam')

['-baba', '-latguuls']

In [282]:
abd ini true
hij ini true
ini lmo true
ini hij true

SyntaxError: invalid syntax (<ipython-input-282-969f8b478aa1>, line 1)

In [283]:
3/4

1.0